In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import random

#シード値の固定
def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

torch_fix_seed()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


データの読み込み

In [2]:
data_path = "/root/work/data/2021"
result_path = "/root/work/result/ViT/depth4"

In [3]:
transform = transforms.Resize((48,512))

x_data=[]

for i in range(1,302):
    img=np.load(file=data_path+"/deck_depth_map/data/hull"+str(i)+".npy")
    img=torch.tensor(img).reshape(1,48,521)
    img=transform(img)

    # img.numpy()
    img=np.array(img[0])
    x_data.append(img)

x_data=np.array(x_data)
x_data = np.delete(x_data, range(117, 119), axis=0)
x_data=np.flip(x_data,axis=2)
x_data=np.array(x_data).reshape(299,1,48,512)
x_data=torch.tensor(x_data)

X_MAX = x_data.max()
x_data /= X_MAX

In [4]:
y =np.genfromtxt(data_path + "/hist_naked.dat", dtype=float, skip_header=1)
y = np.delete(y, range(117, 119), axis=0)
y_data=y[:,4:5]


In [5]:
class HullDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, index):
        return self.data[index], self.targets[index]

In [6]:
import torch
import numpy as np
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.dataset import Subset
from sklearn.model_selection import KFold


batch_size=16

data=torch.tensor(x_data).float().clone().detach()
targets=torch.tensor(y_data).float().clone().detach()

dataset=HullDataset(data,targets)

kfold=KFold(n_splits=5,shuffle=True,random_state=0)

train_loaders=[]
val_loaders=[]

for train_index,val_index in kfold.split(dataset):
    train_dataset=Subset(dataset,train_index)
    val_dataset=Subset(dataset,val_index)

    train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
    val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)

    train_loaders.append(train_loader)
    val_loaders.append(val_loader)



/tmp/ipykernel_2759061/1316001768.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data=torch.tensor(x_data).float().clone().detach()


モデルの読み込み

In [7]:
import timm
import torch.nn as nn


In [8]:
def fit(net, optimizer,num_epochs, train_loader, test_loader, device, history,fold):

    # tqdmライブラリのインポート
    from tqdm.notebook import tqdm

    base_epochs = len(history)
  
    for epoch in range(base_epochs, num_epochs+base_epochs):

        # 1エポックあたりの累積損失(平均化前)
        train_loss, val_loss = 0, 0

        best_val_loss=1.0

        # 1エポックあたりのデータ累積件数
        n_train, n_test = 0, 0

        #訓練フェーズ
        net.train()
        # for inputs, labels in tqdm(train_loader):
        for inputs, labels in train_loader:
            # 1バッチあたりのデータ件数
            train_batch_size = len(labels)
            # 1エポックあたりのデータ累積件数
            n_train += train_batch_size
    
            # GPUヘ転送
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 勾配の初期化
            optimizer.zero_grad()

            # 予測計算
            loss = net(inputs, labels)

            # 勾配計算
            loss.backward()

            # パラメータ修正
            optimizer.step()

            # lossは平均計算が行われているので平均前の損失に戻して加算
            train_loss += loss.item() * train_batch_size 

        #予測フェーズ
        net.eval()

        for inputs_test, labels_test in test_loader:
            # 1バッチあたりのデータ件数
            test_batch_size = len(labels_test)
            # 1エポックあたりのデータ累積件数
            n_test += test_batch_size

            # GPUヘ転送
            inputs_test = inputs_test.to(device)
            labels_test = labels_test.to(device)

            # 予測計算
            loss_test = net(inputs_test, labels_test)

            # lossは平均計算が行われているので平均前の損失に戻して加算
            val_loss +=  loss_test.item() * test_batch_size

        # 損失計算
        rmse_train_loss=(train_loss / n_train)**0.5
        rmse_val_loss=(val_loss / n_test)**0.5     
    

        # 結果表示
        if (epoch+1)%10==0:
             print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}],rmse_train_loss: {rmse_train_loss:.5f},rmse_val_loss: {rmse_val_loss:.5f}')
        # 記録
        item = np.array([epoch+1,
                        rmse_train_loss,rmse_val_loss])

        history = np.vstack((history, item))

        save_path = result_path+'/model'+str(fold)+'.pth'

        if rmse_val_loss<best_val_loss:
            best_val_loss=rmse_val_loss
            # モデルの状態辞書を取得
            model_state = net.state_dict()
            # モデルの状態辞書を保存
            torch.save(model_state, save_path)
        

    return history

In [9]:
from vessel_models.vessel_ViT import vessel_ViT

In [10]:
historys=[]
lr=0.0001
num_epochs=2000
for fold in range(5):
    model = vessel_ViT(depth=4)
    net=model.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    history = np.zeros((0,3))

    train_loader = train_loaders[fold]
    val_loader = val_loaders[fold]

    history = fit(net, optimizer, num_epochs, train_loader, val_loader, device, history,fold)
    historys=np.append(historys,history)

Epoch [10/2000],rmse_train_loss: 0.10877,rmse_val_loss: 0.07523
Epoch [20/2000],rmse_train_loss: 0.07476,rmse_val_loss: 0.04202
Epoch [30/2000],rmse_train_loss: 0.06913,rmse_val_loss: 0.05641
Epoch [40/2000],rmse_train_loss: 0.06825,rmse_val_loss: 0.05021
Epoch [50/2000],rmse_train_loss: 0.06757,rmse_val_loss: 0.05016
Epoch [60/2000],rmse_train_loss: 0.07402,rmse_val_loss: 0.04862
Epoch [70/2000],rmse_train_loss: 0.06946,rmse_val_loss: 0.05113
Epoch [80/2000],rmse_train_loss: 0.06743,rmse_val_loss: 0.04890
Epoch [90/2000],rmse_train_loss: 0.07009,rmse_val_loss: 0.04732
Epoch [100/2000],rmse_train_loss: 0.06994,rmse_val_loss: 0.05113
Epoch [110/2000],rmse_train_loss: 0.06771,rmse_val_loss: 0.04788
Epoch [120/2000],rmse_train_loss: 0.07033,rmse_val_loss: 0.04664
Epoch [130/2000],rmse_train_loss: 0.07113,rmse_val_loss: 0.05719
Epoch [140/2000],rmse_train_loss: 0.06818,rmse_val_loss: 0.06159
Epoch [150/2000],rmse_train_loss: 0.06719,rmse_val_loss: 0.04622
Epoch [160/2000],rmse_train_loss: 

In [11]:
historys=historys.reshape(5,num_epochs,3)

In [12]:
np.save(result_path+'/log', historys)